In [3]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
import numpy as np
from model import DeepHit
from losses import *

In [3]:
df = pd.read_csv("./data/metabric.csv")

In [7]:
scaler = StandardScaler()
df_trans = scaler.fit_transform(df)

In [9]:
df_trans = pd.DataFrame(df_trans, columns=df.columns)

In [19]:
### MASK FUNCTIONS
'''
    mask1      : To calculate LOSS_1 (log-likelihood loss)
    mask2      : To calculate LOSS_2 (ranking loss)
'''
def compute_mask1(time, label, num_Event, num_Category):
    '''
        mask1 is required to get the log-likelihood loss
        mask1 size is [N, num_Event, num_Category]
            if not censored : one element = 1 (0 elsewhere)
            if censored     : fill elements with 1 after the censoring time (for all events)
    '''
    mask = np.zeros([np.shape(time)[0], num_Event, num_Category]) # for the first loss function
    for i in range(np.shape(time)[0]):
        if label[i,0] != 0:  #not censored
            mask[i,int(label[i,0]-1),int(time[i,0])] = 1
        else: #label[i,2]==0: censored
            mask[i,:,int(time[i,0]+1):] =  1 #fill 1 until from the censoring time (to get 1 - \sum F)
    return mask


def compute_mask2(time, meas_time, num_Category):
    '''
        mask2 is required calculate the ranking loss (for pair-wise comparision)
        mask2 size is [N, num_Category].
        - For longitudinal measurements:
             1's from the last measurement to the event time (exclusive and inclusive, respectively)
             denom is not needed since comparing is done over the same denom
        - For single measurement:
             1's from start to the event time(inclusive)
    '''
    mask = np.zeros([np.shape(time)[0], num_Category]) # for the first loss function
    if np.shape(meas_time):  #lonogitudinal measurements
        for i in range(np.shape(time)[0]):
            t1 = int(meas_time[i, 0]) # last measurement time
            t2 = int(time[i, 0]) # censoring/event time
            mask[i,(t1+1):(t2+1)] = 1  #this excludes the last measurement time and includes the event time
    else:                    #single measurement
        for i in range(np.shape(time)[0]):
            t = int(time[i, 0]) # censoring/event time
            mask[i,:(t+1)] = 1  #this excludes the last measurement time and includes the event time
    return mask

In [20]:
time  = np.asarray(df_trans[['event_time']])
# time  = np.round(time/12.) #unit time = month
label = np.asarray(df_trans[['label']])


num_Category    = int(np.max(time) * 1.2)        #to have enough time-horizon
num_Event       = int(len(np.unique(label)) - 1) #only count the number of events (do not count censoring as an event)

x_dim           = df_trans.shape[1]

mask1           = compute_mask1(time, label, num_Event, num_Category)
mask2           = compute_mask2(time, -1, num_Category)

In [21]:
mask2

array([[1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       ...,
       [1., 1., 0., 0.],
       [1., 0., 0., 0.],
       [1., 1., 0., 0.]])

In [ ]:
# Use tf.data API to shuffle and batch data.
train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_data = train_data.repeat().shuffle(5000).batch(batch_size).prefetch(1)

In [ ]:
# Run training for the given number of steps.
for step, (batch_x, batch_y) in enumerate(train_data.take(training_steps), 1):
    # Run the optimization to update W and b values.
    run_optimization(batch_x, batch_y)
    
    if step % display_step == 0:
        pred = custom_net(batch_x, is_training=False)
        loss = cross_entropy(pred, batch_y)
        acc = accuracy(pred, batch_y)
        print("step: %i, loss: %f, accuracy: %f" % (step, loss, acc))

In [22]:
deephit = model.DeepHit()
optimizer = tf.optimizers.Adam(learning_rate)

NameError: name 'model' is not defined

In [ ]:
def run_optimization(x, y, alpha, beta, gamma):
    # Wrap computation inside a GradientTape for automatic differentiation.
    with tf.GradientTape() as g:
        pred = deephit(x)
        loss1 = loss_log_likelihood(pred, y)
        loss2 = loss_ranking(pred, y)
        loss3 = loss_calibration(pred, y)
        total_loss = alpha * loss1 + beta * loss2 + gamma * loss3
        # Compute gradients.
        gradients = g.gradient(total_loss, deephit.trainable_variables)

        # Update W and b following gradients.
        optimizer.apply_gradients(zip(gradients, deephit.trainable_variables))